In [1]:
import sys 
sys.path.append('..')
from ml.inklabel_dataset import InkLabelDataset
import yaml
import lightning as pl
from denoising_diffusion_pytorch import Unet, GaussianDiffusion
import torch
segment_ids = yaml.safe_load(open('../configs/segment_ids.yaml', 'r'))
print(segment_ids['segment_ids'])

SAMPLE_SIZE = 1024

{'segments': ['20231210121321', '20231106155350', '20231005123336', '20230820203112', '20230620230619', '20230826170124', '20230702185753', '20230522215721', '20230531193658', '20230520175435', '20230903193206', '20230902141231', '20231007101615', '20230929220924', 'recto', 'verso', '20231016151000', '20231012184423', '20231031143850']}


/home/npopkov/vesuvius_gcml/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ink_label_dataset = InkLabelDataset(segment_ids['segment_ids']['segments'],SAMPLE_SIZE)
ink_label_dataloader = torch.utils.data.DataLoader(ink_label_dataset, batch_size=16, shuffle=True, num_workers=25)

Creating InkLabelDataset with 19 segments...
Error processing segment verso: invalid literal for int() with base 10: 'verso'Error processing segment recto: invalid literal for int() with base 10: 'recto'

An error occurred while initializing the Volume class: URL not found for scroll: None, energy: None, resolution: None, segment: 20231012184423An error occurred while initializing the Volume class: URL not found for scroll: None, energy: None, resolution: None, segment: 20231031143850

Load the canonical scroll 1 with Volume(type="scroll", scroll_id=1, energy=54, resolution=7.91)Load the canonical scroll 1 with Volume(type="scroll", scroll_id=1, energy=54, resolution=7.91)

If loading another part of the same physical scroll use for instance Volume(type="scroll", scroll_id="1b", energy=54, resolution=7.91)If loading another part of the same physical scroll use for instance Volume(type="scroll", scroll_id="1b", energy=54, resolution=7.91)

Load a segment (e.g. 20230827161847) with Volum

/home/npopkov/vesuvius_gcml/notebooks/../ml/inklabel_dataset.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  self.samples = [torch.tensor(sample, dtype=torch.float32) for sample in self.samples]


Created 3
Created 465 samples from ink labels.


In [11]:
model = Unet(
    dim = 64,
    channels= 1,
    dim_mults = (1, 2, 4,8),
    flash_attn = False
)

diffusion = GaussianDiffusion(
    model,
    image_size = 128,
    timesteps = 1000  
)
class LightningWrapper(pl.LightningModule):
    def __init__(self, diffusion):
        super().__init__()
        self.diffusion = diffusion

    def forward(self, x):
        if len(x.shape) == 3:
            x = x.unsqueeze(1)
        
        return self.diffusion(x)

    def training_step(self, batch, batch_idx):
        x = batch
        # downsample batch to bx128x128
        x = torch.nn.functional.interpolate(x.unsqueeze(1), size=(128, 128), mode='bilinear', align_corners=False)
        # normalize to [0, 1]
        x = (x - x.min()) / (x.max() - x.min())
        pred = self.forward(x)
        print(f"Batch shape: {x.shape}, Pred shape: {pred.shape}")
        loss = torch.nn.functional.mse_loss(pred, x)
        self.log('train_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.diffusion.parameters(), lr=3e-4)
        return optimizer
    
lightning_model = LightningWrapper(diffusion)
trainer = pl.Trainer(max_epochs=1000, accelerator="gpu", devices=1, callbacks=[pl.pytorch.callbacks.early_stopping.EarlyStopping(monitor="train_loss", patience=15, mode="min")])

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [12]:
lightning_model.training_step(next(iter(ink_label_dataloader)),1)
# warm up the dataloader

Batch shape: torch.Size([16, 1, 128, 128]), Pred shape: torch.Size([])


/tmp/ipykernel_229951/513937011.py:32: UserWarning: Using a target size (torch.Size([16, 1, 128, 128])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(pred, x)
/home/npopkov/vesuvius_gcml/.venv/lib/python3.10/site-packages/lightning/pytorch/core/module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(0.2147, grad_fn=<MseLossBackward0>)

In [ ]:
trainer.fit(lightning_model, ink_label_dataloader)
torch.save(lightning_model.state_dict(), 'ink_label_diffusion.pth')
print("Training complete. Model saved as 'ink_label_diffusion.pth'.")